# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing Data set!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing Data again. This time, however, we did things slightly different:
- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [13]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [14]:
age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [3]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(0, 6]","RAD_(6, 24]","TAX_(0, 270]","TAX_(270, 360]","TAX_(360, 712]"
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0


## Run an linear model in Statsmodels

In [15]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
Y= boston_features['CRIM']
x=boston_features[['INDUS', 'CHAS', 'RM', 'AGE', 'DIS', 'PTRATIO', 'B', 'LSTAT']]
formula = "Y ~ x"
model = ols(formula= formula, data=boston_features).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     145.8
Date:                Fri, 10 May 2019   Prob (F-statistic):          3.63e-125
Time:                        00:49:57   Log-Likelihood:                 339.99
No. Observations:                 506   AIC:                            -662.0
Df Residuals:                     497   BIC:                            -623.9
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6423      0.080      7.991      0.000       0.484       0.800
x[0]           0.0658      0.009      6.961      0.000       0.047       0.084
x[1]          -0.0064      0.023     -0.285      0.776      -0.051       0.038
x[2]           0.0216      0.011      1.910      0.057      -0.001       0.044
x[3]           0.0171      0.010      1.730      0.084      -0.002       0.036
x[4]          -0.3373      0.046     -7.367      0.000      -0.427      -0.247
x[5]           0.0208      0.006      3.244      0.001       0.008       0.033
x[6]          -0.1979      0.027     -7.465      0.000      -0.250      -0.146
x[7]           0.0287      0.010      2.868      0.004       0.009       0.048
==============================================================================
Omnibus:                       22.288   Durbin-Watson:                   0.380
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               24.011
Skew:                          -0.520   Prob(JB):                     6.11e-06
Kurtosis:                       3.240   Cond. No.                         96.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
from sklearn import linear_model as lm

Y= boston_features['CRIM']
model=lm.LinearRegression()
results=model.fit(Y,x)
results.summary()

ValueError: Expected 2D array, got 1D array instead:
array=[0.         0.15321102 0.15313433 0.1710054  0.25031515 0.16252092
 0.27604647 0.32765636 0.36737091 0.34465818 0.37392596 0.3059401
 0.28236161 0.48172367 0.48307797 0.48132896 0.53563078 0.50468389
 0.50712614 0.4965823  0.55364179 0.51336959 0.55201348 0.52891378
 0.50005215 0.51194703 0.4885058  0.52539628 0.50317663 0.53038822
 0.5430015  0.56191408 0.56445395 0.54491958 0.58017015 0.2426422
 0.28636952 0.26590748 0.34769803 0.15443053 0.17487839 0.31446808
 0.32542386 0.33786744 0.31049161 0.34550435 0.35536975 0.37594524
 0.38661501 0.37151506 0.27656688 0.20162972 0.22380017 0.21612322
 0.08022593 0.07638454 0.12343925 0.08562639 0.33459126 0.29246295
 0.33105511 0.3456813  0.29931862 0.31369305 0.11800253 0.18166581
 0.20263863 0.23186051 0.32091893 0.31505786 0.27601089 0.33747254
 0.27994507 0.3592057  0.2643546  0.28384685 0.29067393 0.27458982
 0.22924208 0.27066993 0.19607822 0.20460428 0.18383389 0.18069744
 0.21

## Run the same model in Scikit-learn

In [24]:
import numpy as np
from sklearn.linear_model import LinearRegression
Y= boston_features['CRIM']
reg = LinearRegression().fit(Y, x)

ValueError: Expected 2D array, got 1D array instead:
array=[0.         0.15321102 0.15313433 0.1710054  0.25031515 0.16252092
 0.27604647 0.32765636 0.36737091 0.34465818 0.37392596 0.3059401
 0.28236161 0.48172367 0.48307797 0.48132896 0.53563078 0.50468389
 0.50712614 0.4965823  0.55364179 0.51336959 0.55201348 0.52891378
 0.50005215 0.51194703 0.4885058  0.52539628 0.50317663 0.53038822
 0.5430015  0.56191408 0.56445395 0.54491958 0.58017015 0.2426422
 0.28636952 0.26590748 0.34769803 0.15443053 0.17487839 0.31446808
 0.32542386 0.33786744 0.31049161 0.34550435 0.35536975 0.37594524
 0.38661501 0.37151506 0.27656688 0.20162972 0.22380017 0.21612322
 0.08022593 0.07638454 0.12343925 0.08562639 0.33459126 0.29246295
 0.33105511 0.3456813  0.29931862 0.31369305 0.11800253 0.18166581
 0.20263863 0.23186051 0.32091893 0.31505786 0.27601089 0.33747254
 0.27994507 0.3592057  0.2643546  0.28384685 0.29067393 0.27458982
 0.22924208 0.27066993 0.19607822 0.20460428 0.18383389 0.18069744
 0.21

## Remove the necessary variables to make sure the coefficients are the same for Scikit-learn vs Statsmodels

### Statsmodels

### Scikit-learn

## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing Data.